# 线性回归的简洁实现

In [1]:
# 生成数据集
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)


In [3]:
# 读取数据集
def load_array(data_arrays, batch_size, is_train=True): #@save
    """ 构造一个pytorch数据迭代器 """
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [4]:
# 使用iter构造Python迭代器，并使用next从迭代器中获取第一项。
next(iter(data_iter))

[tensor([[-0.3293,  1.5187],
         [ 0.5213, -0.6235],
         [ 0.0094,  0.3800],
         [ 0.4887, -1.1688],
         [-0.8526,  0.6205],
         [ 1.3919, -0.8455],
         [-0.3936,  0.1308],
         [ 0.3048,  1.1150],
         [-0.0999,  0.3057],
         [ 0.3396, -1.3713]]),
 tensor([[-1.6233],
         [ 7.3605],
         [ 2.9381],
         [ 9.1577],
         [ 0.3940],
         [ 9.8724],
         [ 2.9631],
         [ 1.0120],
         [ 2.9485],
         [ 9.5461]])]

In [83]:
# 定义模型
# nn是neutral network神经网络的缩写
# 我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状(X)，即2，第二个指定输出特征形状(y)，输出特征形状为单个标量，因此为1。
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

In [102]:
# 初始化模型参数
# 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [110]:
# 定义损失函数
# 计算均方误差使用的是MSELoss类，也称为平方L2范数。 默认情况下，它返回所有样本损失的平均值。
loss = nn.MSELoss()
# loss = nn.HuberLoss()
# loss = nn.MSELoss(reduction='sum')

In [111]:
# 定义优化算法
# 小批量随机梯度下降算法
"""
指定优化的参数 （可通过net.parameters()从我们的模型中获得）
"""
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [112]:
# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000100
epoch 2, loss 0.000100
epoch 3, loss 0.000100


In [65]:
# 比较参数误差
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0002, -0.0005])
b的估计误差： tensor([-0.0001])


In [73]:
net[0].bias.grad

tensor([-0.0158])